# Data Cleaning
---


**Objective:** to get the data for the following information for all countries from December 1959 to December 1990


1. Industrial production (Index)

2. Exchange rates, National Currency per US dollar (Period Average)

3. Consumer prices (All items), index

4. International Reserves and Liquidity (Reserves, Official Reserve Assets, US Dollar)

5. Data for consumer prices and international reserves for the United States only over the same time period.


---

# 1. Downloading the data

We will be collecting the data from ['IMF data portal'](https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sid=1390030341854) using the query function to get desired data